Cleaning ideas
* province

* postal code
* area code
* city names
* those fix mes

# Library imports and file definitions

In [17]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import codecs
import json

sample = True
#sample = False

if sample:
    print "Using sample"
    osmfile = 'sample.osm'
    jsonfile = osmfile + ".json"  
else:
    #osmfile = '~/Google Drive/Data Science/Udacity/Project3.Data Wrangling with MongoDB/vancouver_canada.osm'
    print "Using complete file"
    osmfile = 'vancouver_canada.osm'
    jsonfile = osmfile + ".json"
print osmfile
print jsonfile
filename = osmfile

Using sample
sample.osm
sample.osm.json


# Code for creating sample

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "vancouver_canada.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 20 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# Key Types

In [4]:
"""
Your task is to explore the data a bit more.
Before you process the data and add it into MongoDB, you should check the "k"
value for each "<tag>" and see if they can be valid keys in MongoDB, as well as
see if there are any other potential problems.

We have provided you with 3 regular expressions to check for certain patterns
in the tags. As we saw in the quiz earlier, we would like to change the data
model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with
problematic characters.

Please complete the function 'key_type', such that we have a count of each of
four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        #pass        '''
        key = element.get("k") 
        #print key
        if re.match(lower, key): #match starting from beginning of string
            keys["lower"] += 1
            #keys["lower"].append(key)
            #print key
        elif re.match(lower_colon, key): 
            keys["lower_colon"]  += 1
            #keys["lower_colon"].append(key)
            #print key
        elif  re.search(problemchars, key): #search anywhere
            keys["problemchars"]  += 1
            #keys["problemchars"].append(key)
             #print key
        else:
            keys["other"]  += 1

                #keys["other"].append(key) 
                #print key       
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



In [5]:
process_map(osmfile)

{'lower': 235553, 'lower_colon': 25803, 'other': 2851, 'problemchars': 2}

# Users

In [6]:
def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        pass
        if element.tag == "node":
            user = element.get("user") 
            if user not in users:
                users.add(user) #not users.append(user) because users is a set, not a dict

    return users



In [8]:
users = process_map(osmfile)
pprint.pprint(users)

set([None,
     '2bre99',
     '4b696d',
     '7503146',
     '>LAND',
     'ACervantes',
     'AKC',
     'Aaron Hilton',
     'Achilnos',
     'Adam Dunn',
     'AdamWill',
     'Alan',
     'Alan Trick',
     'Aleks-Berlin',
     'Alex Cruise',
     'AmateurCartographer',
     'AndiG88',
     'Andre68',
     'Andrea428',
     'AndyGibb',
     'Anthony Steele',
     'Anton Lavrov',
     'Aquin',
     'Art Olin',
     'Arthur Chan',
     'BC Hiker',
     'BCBiker',
     'Bellbellbell',
     'Blaireo',
     'BlaneG',
     'Blobo123',
     'Bootprint',
     'Brandon5228',
     'BruceMap',
     'Bryce C Nesbitt',
     'BuganiniQ',
     'Bullroarer',
     'Burrito!',
     'C2H0',
     'CBDVN',
     'CJ60',
     'Cababunga',
     'CamD',
     'CamJW',
     'CamNZ',
     'CandyFu',
     'ChrissW-R1',
     'Cicerone',
     'Claudius Henrichs',
     'CloCkWeRX',
     'Cobra',
     'Concession',
     'CoreyBurger',
     'Creando',
     'DENelson83',
     'DKazzed',
     'DMDK',
     'DPTRob',


# Auditing Street Types

In [11]:
OSMFILE = filename
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Alley", "Broadway", "Walk","Crescent", "Esplanade", "Highway","Kingsway","Mews","Mall","Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "South","Commons","Way","West","East"]

# UPDATE THIS VARIABLE
mapping = { "Blvd": "Boulevard",
            "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Steet": "Street",
            "street": "Street",
            "venue": "Avenue",
            "Broughton": "Broughton Street",
            "Jervis": "Jervis Street",
            "Jarvis": "Jervis Street"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):

    # YOUR CODE HERE
    for key, value in mapping.iteritems():
        key_at_end = re.escape(key) + r"$"
        name = re.sub(key_at_end, value, name)

    return name



In [3]:
st_types = audit(osmfile)
pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)

{}


# Auditing City Names

In [7]:

city_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ['Vancouver']

# UPDATE THIS VARIABLE
mapping = { "Vancovuer": "Vancouver",
            "vancouver": "Vancouver",
            }


def audit_city_type(city_types, city_name):
    m = city_type_re.search(city_name)
    if m:
        city_type = m.group()
        if city_type not in expected:
            city_types[city_type].add(city_name)


def is_city_name(elem):
    return (elem.attrib['k'] == "addr:city")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    city_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city_name(tag):
                    audit_city_type(city_types, tag.attrib['v'])
    osm_file.close()
    return city_types


def update_name(name, mapping):

    # YOUR CODE HERE
    for key, value in mapping.iteritems():
        key_at_end = re.escape(key) + r"$"
        name = re.sub(key_at_end, value, name)

    return name

#'''
st_types = audit(osmfile)
pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
#'''

{'Vancovuer': set(['Vancovuer']),
 'vancouver': set(['vancouver']),
 u'\u0422\u0440\u0435\u0442\u0438\u0439 \u0421\u0435\u0432\u0435\u0440\u043d\u044b\u0439': set([u'\u0422\u0440\u0435\u0442\u0438\u0439 \u0421\u0435\u0432\u0435\u0440\u043d\u044b\u0439'])}


# Auditing Province Name

In [16]:

city_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ['British Columbia']

# UPDATE THIS VARIABLE
mapping = { "B.C.": "British Columbia",
            "bc": "British Columbia",
            }


def audit_city_type(city_types, city_name):
    #m = var_type_re.search(var_name)
    m = city_name
    if m:
        #var_type = m.group()
        city_type = city_name
        if city_type not in expected:
            city_types[city_type].add(city_name)


def is_city_name(elem):
    return (elem.attrib['k'] == "addr:province")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    city_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city_name(tag):
                    audit_city_type(city_types, tag.attrib['v'])
    osm_file.close()
    return city_types


def update_name(name, mapping):

    # YOUR CODE HERE
    for key, value in mapping.iteritems():
        key_at_end = re.escape(key) + r"$"
        name = re.sub(key_at_end, value, name)

    return name

#'''
st_types = audit(osmfile)
pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
#'''

{'B.C.': set(['B.C.']),
 'British Columbia': set(['British Columbia']),
 'Cavite': set(['Cavite']),
 'Kent': set(['Kent']),
 'ON': set(['ON']),
 'bc': set(['bc']),
 u'\u611b\u77e5\u770c': set([u'\u611b\u77e5\u770c'])}


# Auditing Postal Codes

In [19]:

var_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected_postalcodes_re = re.compile(r'V[5|6][A-Z] [0-9][A-Z][0-9]$')


# UPDATE THIS VARIABLE
mapping = { "Vancovuer": "Vancouver",
            "vancouver": "Vancouver",
            }


def audit_var_type(var_types, var_name):
    #m = var_type_re.search(var_name)
    m = var_name
    if m:
        #var_type = m.group()
        var_type = var_name
        if not expected_postalcodes_re.search(var_name):
            var_types[var_type].add(var_name)


def is_var_name(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    var_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_var_name(tag):
                    audit_var_type(var_types, tag.attrib['v'])
    osm_file.close()
    return var_types


def update_name(name, mapping):

    # YOUR CODE HERE
    
    name = name.upper()
    no_space_re = re.compile(r'V[5|6][A-Z][0-9][A-Z][0-9]$')
    if no_space_re.search(name):
        #insert a space into postal code
        name = name[:3] + " " + name[3:]

    return name

#'''
st_types = audit(osmfile)
pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, better_name
#'''

{'V5K3K3': set(['V5K3K3']),
 'V5V3R5': set(['V5V3R5']),
 'V5Z2R3': set(['V5Z2R3']),
 'V6E2P4': set(['V6E2P4']),
 'V6H2R2': set(['V6H2R2']),
 'V6M1M3': set(['V6M1M3']),
 'V6M3A5': set(['V6M3A5']),
 'V6T1Z1': set(['V6T1Z1']),
 'V6Z1R2': set(['V6Z1R2']),
 'V6Z2T9': set(['V6Z2T9']),
 'v5y 3w3': set(['v5y 3w3'])}
V6E2P4 V6E 2P4
V6Z1R2 V6Z 1R2
V5Z2R3 V5Z 2R3
V6M3A5 V6M 3A5
V6Z2T9 V6Z 2T9
V6T1Z1 V6T 1Z1
V5K3K3 V5K 3K3
V5V3R5 V5V 3R5
V6M1M3 V6M 1M3
v5y 3w3 V5Y 3W3
V6H2R2 V6H 2R2


# Code for printing out file

In [17]:
def shape_element(element):
    print element
    if element.tag == "node" or element.tag == "way" or element.tag == "relation" :
        for key, value in element.attrib.iteritems():
            print 'key', 'value', key, value
        

        for child in element:
            print 'child', child
            for key, value in child.attrib.iteritems():
                print "key, value", key, value

# Code for preparing for MongoDB

In [28]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        this_created = {}
        node['type'] = element.tag
        # based on https://discussions.udacity.com/t/my-code-is-not-finding-a-specific-tag-attribute/157674/5
        
        for key, value in element.attrib.iteritems():
            #attributes in the CREATED array should be added under a key "created"
            #attributes for latitude and longitude should be added to a "pos" array,for use in geospacial indexing. Make sure the values inside "pos" array are floats and not strings. 
            created_list = ['version','changeset','timestamp','user','uid']
            if key in created_list:
                this_created[key] = element.attrib[key]
            
            elif key == 'lon':
                pass      # will be dealt with in 'lat' 
            elif key == 'lat': 
                node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]          
            else:
                node[key] = value
        node['created'] = this_created
        if element.tag == "way":
            node['node_refs'] = []
                
            #if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
        this_address = {}
        keyfornode = ""
        for child in element: #note: not an interior loop of above, though 
            for key, value in child.attrib.iteritems():
                #if the second level tag "k" value contains problematic characters, it should be ignored
                if 'k' in child.attrib: #make sure that there is a key tag
                    if  re.search(problemchars, child.attrib['k']):     
                        #pass
                        print "k Has problem chars: ", child.attrib['k']
                    elif re.match('addr:',child.attrib['k']): #if k tag starts with addr:
                        if child.attrib['k'].count(':') > 1: #if k tag has 2 colons, ignore
                            pass
                        else:
                            temp = child.attrib['k'].split(':')[1]
                            this_address[temp] = child.attrib['v']
                            #print this_address
                    
                    elif child.attrib['k'].count(':') > 0: #if k tag does not start with addr: but has a colon
                        #pass
                        
                        #print 'k Has colon: ', child.attrib['k']
                        if child.attrib['k'].count(':') > 1: #if k tag has 2 colons, ignore
                            pass
                        else:
                            # capture the content before the colon
                            if keyfornode == "" or keyfornode != child.attrib['k'].split(':')[0]:
                                keyfornode = child.attrib['k'].split(':')[0]
                                this_other = {}
                            subkey = child.attrib['k'].split(':')[1]
                            this_other[subkey] = child.attrib['v']
                            node[keyfornode] = this_other #update
                            #print this_address
                    else:
                        #pass
                        #print 'k is something else: ', child.attrib['k']
                        node[child.attrib['k']] = child.attrib['v']
                elif 'ref' in child.attrib:
                    node['node_refs'].append(child.attrib['ref'])
                    #print node['node_refs']
                elif 'v' not in child.attrib:
                     print 'the child.attrib is something other than k, v, or ref: ', child.attrib
                   
                else:
                    pass
        if this_address: #if this_address is not empty
            node['address'] = this_address

        
        #print node
        return node
    else:
        return None




In [ ]:
# Code for dumping to JSON

In [30]:
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


In [31]:
data = process_map(osmfile, False)
#need to send this to a file


k Has problem chars:  Toll Free
k Has problem chars:  Toll Free
k Has problem chars:  addr.source:city
k Has problem chars:  addr.source:city


# MongoDB

In

In [2]:
"""
Your task is to sucessfully run the exercise to see how pymongo works
and how easy it is to start using it.
You don't actually have to change anything in this exercise,
but you can change the city name in the add_city function if you like.

Your code will be run against a MongoDB instance that we have provided.
If you want to run this code locally on your machine,
you have to install MongoDB (see Instructor comments for link to installation information)
and uncomment the get_db function.

Note: need to start mongodb in another terminal
"""

def insert_geo(infile, db):
    data = process_file(infile)
    # Add your code here. Insert the data in one command.
    for d in data:
        db.autos.insert(d)
    
def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.examples
    return db

if __name__ == "__main__":
    # For local use
    db = get_db() # uncomment this line if you want to run this locally
    add_city(db)
    print get_city(db)

/Users/DerekAtWork/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{u'_id': ObjectId('570eb3418e573428046dfe6a'), u'name': u'Chicago'}


In [1]:
#!/usr/bin/env python
"""

Your task is to complete the 'porsche_query' function and in particular the query
to find all autos where the manufacturer field matches "Porsche".
Please modify only 'porsche_query' function, as only that will be taken into account.

Your code will be run against a MongoDB instance that we have provided.
If you want to run this code locally on your machine,
you have to install MongoDB and download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials at
the following link:
https://www.udacity.com/wiki/ud032
"""

#replicate > db.yvr.find({"address.street" : "West Broadway"}).count()



def bway_query():
    # Please fill in the query to find all autos manuafactured by Porsche.
    query = {"address.street" : "West Broadway"}
    return query


# Do not edit code below this line in the online code editor.
# Code here is for local use on your own computer.
def get_db(db_name):
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def find_yvr(db, query):
    # For local use
    return db.yvr.find(query)


if __name__ == "__main__":
    # For local use
    db = get_db('geo')
    query = bway_query()
    results = find_yvr(db, query)

    print "Printing first 3 results\n"
    import pprint
    for street in results[:3]:
        pprint.pprint(street)

Printing first 3 results

{u'_id': ObjectId('571562eaa092e06eee9be2e6'),
 u'address': {u'housenumber': u'457',
              u'postcode': u'V5Y 1R4',
              u'street': u'West Broadway'},
 u'created': {u'changeset': u'26984530',
              u'timestamp': u'2014-11-23T22:51:29Z',
              u'uid': u'1408522',
              u'user': u'Omnific',
              u'version': u'6'},
 u'id': u'567129443',
 u'pos': [49.2633391, -123.11437],
 u'type': u'node'}
{u'_id': ObjectId('571562eaa092e06eee9be2e7'),
 u'address': {u'housenumber': u'453',
              u'postcode': u'V5Y 1R4',
              u'street': u'West Broadway'},
 u'created': {u'changeset': u'20153277',
              u'timestamp': u'2014-01-23T03:19:05Z',
              u'uid': u'1891976',
              u'user': u'AdamWill',
              u'version': u'6'},
 u'id': u'567130209',
 u'pos': [49.263337, -123.1142702],
 u'type': u'node'}
{u'_id': ObjectId('571562eaa092e06eee9be2e8'),
 u'address': {u'housenumber': u'445',
       

In [6]:
#!/usr/bin/env python

def get_db(db_name):
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

if __name__ == "__main__":
    # For local use
    db = get_db('geo')
    query = db.vyr.find().count()
    results = find_yvr(db, query)

    print "Printing first 3 results\n"
    import pprint
    print results
    #for street in results[:3]:
        #pprint.pprint(street)




TypeError: filter must be an instance of dict, bson.son.SON, or other type that inherits from collections.Mapping

In [7]:
#at mongo prompt:
mongoimport --db geo -c yvr --file vancouver_canada.osm.json

SyntaxError: invalid syntax (<ipython-input-7-b7993fd1b35c>, line 2)